In [25]:
from tqdm.notebook import tqdm
import codecs
import pandas as pd
tqdm.pandas()
import string
from multiprocessing import  Pool
import numpy as np
import random

In [26]:
from google.colab import drive
import os
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
os.listdir()

['codes',
 'dataset',
 'openweiboscope',
 'unzipData',
 'testNew.csv',
 'FinalDataset',
 'Ecodeing and saving.ipynb',
 'GUSE TEST.ipynb',
 'preprocessed_Data.csv',
 'encodedDatasetV2',
 'splitData',
 'preprocessing.ipynb',
 'tmp',
 'structured_data_classifier']

In [28]:
# os.chdir("./drive/MyDrive/NLP Project")

In [29]:
# %%capture
#@title Setup Environment
# Install the latest Tensorflow version.
!pip install "tensorflow-text==2.8.*" -q
!pip install bokeh -q  
!pip install simpleneighbors[annoy] -q
!pip install tqdm -q

In [30]:
import pandas as pd
import os

In [31]:
from tqdm.notebook import tqdm
import codecs
import pandas as pd
tqdm.pandas()
import string
from multiprocessing import  Pool
import numpy as np

import random
import bokeh
import bokeh.models
import bokeh.plotting
import numpy as np
import os
import pandas as pd
import tensorflow.compat.v2 as tf
import tensorflow_hub as hub
from tensorflow_text import SentencepieceTokenizer
import sklearn.metrics.pairwise

from simpleneighbors import SimpleNeighbors
from tqdm import tqdm
from tqdm import trange

In [32]:

def parallelize_dataframe(df, func, n_cores=2):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [33]:
os.listdir()

['codes',
 'dataset',
 'openweiboscope',
 'unzipData',
 'testNew.csv',
 'FinalDataset',
 'Ecodeing and saving.ipynb',
 'GUSE TEST.ipynb',
 'preprocessed_Data.csv',
 'encodedDatasetV2',
 'splitData',
 'preprocessing.ipynb',
 'tmp',
 'structured_data_classifier']

In [34]:
path="./splitData"
# os.chdir("./drive/MyDrive/NLP Project")


In [35]:
# os.listdir(path)

In [36]:
df_censored=pd.read_csv(path+"/censored.csv",names=["mid","text","created_at","deleted_last_seen","permission_denied"
]).dropna()

In [37]:
df_censored = df_censored.drop(df_censored.index[df_censored['text'] == "text"])

In [38]:
len(df_censored)

38965

In [39]:
cols=["mid","text","created_at","deleted_last_seen","permission_denied"]

In [40]:
df_uncensored=pd.read_csv(path+"/not_censored.csv",names=["mid","text","created_at","deleted_last_seen","permission_denied"
],nrows=38965*2).dropna()

In [41]:
# The 16-language multilingual module is the default but feel free
# to pick others from the list and compare the results.
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual/3' #@param ['https://tfhub.dev/google/universal-sentence-encoder-multilingual/3', 'https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3']

model = hub.load(module_url)

def embed_text(input):
    return model(input)[-1]

def embed_text_list(input):
    return model(input).numpy()

In [42]:
def parallel_guse(df):
    #df["guse"] = df["text"].progress_apply(lambda x:embed_text(str(x)))
    df["guse"] = embed_text_list(df["text"].to_list()).tolist()
    return df

In [43]:
# df_test=parallelize_dataframe(df=df_censored[:200], func=parallel_guse, n_cores=6)

In [44]:
# parallel_guse(df_censored[:200])

In [45]:
def serial_dataframe(df, func, n_splits=2):
    df_split = np.array_split(df, n_splits)
    df = pd.DataFrame()
    for df_s in tqdm(df_split):
        df_rs=func(df_s)
        df = pd.concat([df,df_s])
    return df

In [46]:
df_censored=serial_dataframe(df=df_censored, func=parallel_guse, n_splits=20)

100%|██████████| 20/20 [00:08<00:00,  2.30it/s]


In [47]:
df_uncensored=serial_dataframe(df=df_uncensored, func=parallel_guse, n_splits=20*2)

100%|██████████| 40/40 [00:13<00:00,  3.06it/s]


In [48]:
df_uncensored.head()

,mid,text,created_at,deleted_last_seen,permission_denied,guse
0,mCClUNCqwe,转发微博,2012-01-03 02:02:27,False,False,"[0.10424912720918655, 0.04464416205883026, 0.0..."
1,mRsOcOLTlc,uK3RXUYW3： u0AGMTTVD： ！！！！！！！！,2012-01-03 01:17:39,False,False,"[-0.004717760719358921, -0.06476746499538422, ..."
2,mH44qG6iUm,求一切順利,2012-01-03 01:15:36,False,False,"[0.0962323322892189, -0.005889754742383957, 0...."
3,mZmwFtOdVX,想要uK3RXUYW3： ukn： 全都想要啊QAQ,2012-01-03 01:12:55,False,False,"[0.00019058313046116382, -0.02642415463924408,..."
4,mQkLJSl8bf,ukn： uMLLV3ZCO： 转发微博,2012-01-03 01:10:42,False,False,"[0.015181043185293674, 0.013890189118683338, 0..."


In [49]:
df_data=pd.concat([df_censored.loc[:,['permission_denied','guse']],df_uncensored.loc[:,['permission_denied','guse']]])

In [50]:
df_data["permission_denied"]=df_data["permission_denied"].astype(bool).astype(int)

In [51]:
X=df_data["guse"].to_numpy()
X=np.vstack(X)

In [52]:
y=df_data["permission_denied"].values

In [53]:
y.shape

(116725,)

In [54]:
# Y

In [55]:
# from pprint import pprint

# import sklearn.datasets
# import sklearn.metrics

# import autosklearn.classification

In [56]:
# !pip install git+https://github.com/hyperopt/hyperopt-sklearn.git

In [57]:
# df_data.to_pickle(path+"/embeding_data.pickle")

In [58]:
path

'./splitData'

In [59]:
# !pip3 install auto-sklearn


In [60]:
# import autosklearn.classification

In [61]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [62]:


test_size = int( 0.2 * len( y ) )
# np.random.seed( seed )
# np.random.RandomState(2)
indices = np.random.permutation(len(X))
X_train = X[ indices[:-test_size]]
y_train = y[ indices[:-test_size]]
X_test = X[ indices[-test_size:]]
y_test = y[ indices[-test_size:]]


In [63]:
# automl = autosklearn.classification.AutoSklearnClassifier(
#     time_left_for_this_task=1200,
#     per_run_time_limit=30
# )

In [64]:
# automl.fit(X_train, y_train)

In [65]:
# automl.show_models()

In [66]:
# !pip install hyperopt

In [67]:
!pip install hpsklearn
# !pip install -U scikit-learn scipy matplotlib

  Using cached nose-1.3.7-py3-none-any.whl (154 kB)
  Created wheel for hpsklearn: filename=hpsklearn-0.1.0-py3-none-any.whl size=23913 sha256=88a1601311af616d05fd9808f890dcd966055f74779d3c30a7ede9502a68eefe
  Stored in directory: /root/.cache/pip/wheels/b2/22/23/9207fbe861f70774b563f909a415b228450bfcf863c64ac1e8
Successfully built hpsklearn


In [68]:
from hpsklearn import HyperoptEstimator, any_classifier
# from sklearn.datasets import fetch_mldata
from hyperopt import tpe
import numpy as np
from hpsklearn import HyperoptEstimator, svc, knn,random_forest,svc_rbf
from hyperopt import hp

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [69]:
# clf = hp.choice('my_name',[random_forest('my_name.random_forest'),svc('my_name.svc'),knn('my_name.knn'),svc_rbf('my_name.svc_rbf')])

# estim = HyperoptEstimator(classifier=clf, algo=tpe.suggest,max_evals=150,trial_timeout=60)

In [70]:



test_size = int( 0.2 * len( y ) )
# np.random.seed( seed )
# np.random.RandomState(2)
indices = np.random.permutation(len(X))
X_train = X[ indices[:-test_size]]
y_train = y[ indices[:-test_size]]
X_test = X[ indices[-test_size:]]
y_test = y[ indices[-test_size:]]

# estim.fit(X_train, y_train)
# print(estim.score(X_test, y_test))
# print(estim.best_model())

In [71]:
# https://github.com/albazahm/Credit_Card_Fraud_Detection_with_XGBoost_and_HyperOpt/blob/master/XGBoost%20%2B%20Hyperopt%20in%20Credit%20Card%20Fraud%20Detection.ipynb

In [72]:
#loading libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, make_scorer, confusion_matrix, classification_report, precision_recall_curve, plot_precision_recall_curve, average_precision_score, auc
from sklearn.model_selection import train_test_split
import seaborn as sns
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
import xgboost as xgb
# import shap
# Any results you write to the current directory are saved as output.

In [73]:
# defining the space for hyperparameter tuning
space = {'eta': hp.uniform("eta", 0.1, 1),
        'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 50, 200, 1),
        'reg_lambda' : hp.uniform('reg_lambda', 0, 1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5, 1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': hp.quniform('n_estimators', 100, 200, 10)
        }

In [74]:
#defining function to optimize
def hyperparameter_tuning(space):
    clf = xgb.XGBClassifier(n_estimators = int(space['n_estimators']),       #number of trees to use
                            eta = space['eta'],                              #learning rate
                            max_depth = int(space['max_depth']),             #depth of trees
                            gamma = space['gamma'],                          #loss reduction required to further partition tree
                            reg_alpha = int(space['reg_alpha']),             #L1 regularization for weights
                            reg_lambda = space['reg_lambda'],                #L2 regularization for weights
                            min_child_weight = space['min_child_weight'],    #minimum sum of instance weight needed in child
                            colsample_bytree = space['colsample_bytree'],    #ratio of column sampling for each tree
                            nthread = -1)                                    #number of parallel threads used
    
    evaluation = [(X_train, y_train), (X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set = evaluation,
            early_stopping_rounds = 5,
            verbose = False)

    pred = clf.predict(X_test)
    pred = [1 if i>= 0.5 else 0 for i in pred]
    f1 = f1_score(y_test, pred)
    print ("SCORE:", f1)
    return {'loss': -f1, 'status': STATUS_OK }

In [ ]:
# run the hyper paramter tuning
trials = Trials()
best = fmin(fn = hyperparameter_tuning,
            space = space,
            algo = tpe.suggest,
            max_evals = 20,
            trials = trials)

print (best)

SCORE:
0.726230753296969
SCORE:
0.7365013774104683
 10%|█         | 2/20 [29:39<4:32:21, 907.88s/it, best loss: -0.7365013774104683]

In [ ]:
data={"trials":trials,"best":best}

In [ ]:
import pickle
with open(path+'/XGBClassifier_data.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# with open(path+'/estim.pickle', 'rb') as handle:
#     estim = pickle.load(handle)

In [ ]:
import os

scores = {} # scores is an empty dict already
target=path+'/estim.pickle'
if os.path.getsize(path+'/estim.pickle') > 0:      
    with open(target, "rb") as f:
        unpickler = pickle.Unpickler(f)
        # if file is not empty scores will be equal
        # to the value unpickled
        scores = unpickler.load()